In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import lzma
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

import imageio

from scipy.optimize import basinhopping
from scipy.ndimage import median_filter
from scipy.stats import linregress
from scipy.interpolate import RegularGridInterpolator

import pydicom

from pymedphys.film import (
    calc_calibration_points, load_cal_scans, 
    create_dose_function, calc_net_od, create_axes)

from pymedphys.dicom import xyz_axes_from_dataset, dose_from_dataset

In [ ]:
data_dir = Path('../../packages/pymedphys_analysis/tests/film/data/spine_case/DatasetB')

calibration_alignments = {650.0: (-0.03693119837922606, -0.0947635537173132, -0.37783221426724356),
 200.0: (-0.10040063983494804, -0.12169175795144278, 0.08348855748332341),
 1250.0: (-0.3032988411889664, 0.03641656353978792, 0.18968972666517137),
 1000.0: (0.26492510731092084, 0.14060820397579227, -2.7193448468631343),
 400.0: (-0.08179282402379366, -0.0679576959847175, 0.3154216316864221),
 800.0: (0.18816305442327136, 0.07207526506185861, -0.9586838845463561),
 0.0: (-0.09484272563570335, 0.12621856065409, -0.7927960758287966),
 550.0: (-0.21896673466613484, -0.3278034483394189, -2.275540688446474),
 300.0: (-0.06648768945080456, -0.0075448105213691595, -0.4383480611779579),
 1400.0: (0.21542726392752917, 0.2342317915525293, 0.031818297008679844),
 700.0: (0.04596862036224883, -0.1862675492795798, -0.5631021348448653),
 620.0: (0.045973899659790485, 0.008418383396753638, 0.056398167284774),
 500.0: (0.0157791792147568, 0.14108835739889178, -0.7097770417159057),
 900.0: (-0.1973165785177423, 0.06716167826823238, 0.09585676472845489),
 600.0: (0.0655280798391002, 0.010254562005567824, -0.04182894414171261),
 750.0: (-0.13149497704528673, 0.08891070049242197, 1.0078733699651568),
 580.0: (0.09292906883636731, -0.18062150972708307, -0.3395879590451904),
 1100.0: (-0.05951520624559155, -0.1508183573554567, 0.2423935149180743)}


# alignments = None  # Uncomment this line to recalc the alignments (slow)

treatment_alignment = (-0.09021456658200089, -0.1541382806538466, -0.07098367045728018)

# treatment_alignment = None  # Uncomment this line to recalc the alignment (slow)

In [ ]:
prescans_dir = data_dir.joinpath('prescans/calibration')
postscans_dir = data_dir.joinpath('postscans/calibration')

In [ ]:
treatment_prescan_filepath = data_dir.joinpath('prescans/treatment.tif')
treatment_postscan_filepath = data_dir.joinpath('postscans/treatment.tif')
treatment_lasers_filepath = data_dir.joinpath('postscans/treatment_with_laser_markers.tif')

In [ ]:
dose_zip_path = next(data_dir.glob('../Raw/*.dcm.xz'))
                     
with lzma.open(dose_zip_path) as a_file:
    dicom_dose_dataset = pydicom.dcmread(a_file, force=True)

In [ ]:
prescans = load_cal_scans(prescans_dir)
postscans = load_cal_scans(postscans_dir)

In [ ]:
points, alignments = calc_calibration_points(prescans, postscans, alignments=calibration_alignments, figures=True, pixel_trim=60)
# calibration_alignments

In [ ]:
dose_cal = []
net_od_cal = []

for key, item in points.items():
    net_od_cal.append(item)
    dose_cal.append(key)

In [ ]:
index = np.argsort(net_od_cal)
dose_cal = np.array(dose_cal)[index]
net_od_cal = np.array(net_od_cal)[index]

In [ ]:
dose_function = create_dose_function(net_od_cal, dose_cal)

In [ ]:
x = np.linspace(np.min(net_od_cal), np.max(net_od_cal))
y = dose_function(x)

plt.plot(net_od_cal, dose_cal, '.')
plt.plot(x, y)

In [ ]:
treatment_prescan = imageio.imread(treatment_prescan_filepath)
treatment_postscan = imageio.imread(treatment_postscan_filepath)

In [ ]:
treatment_net_od, treatment_alignment = calc_net_od(
    treatment_prescan, treatment_postscan, alignment=treatment_alignment)
# treatment_alignment

In [ ]:
treatment_dose = dose_function(treatment_net_od)

make_nan = treatment_dose > np.max(dose_cal)*1.2
treatment_dose[make_nan] = np.nan

In [ ]:
plt.figure(figsize=(10,10))
plt.pcolormesh(treatment_dose)
plt.colorbar()
plt.axis('equal')

In [ ]:
filtered_dose = median_filter(treatment_dose, size=10)

In [ ]:
plt.figure(figsize=(10,10))
plt.pcolormesh(filtered_dose)
plt.colorbar()
plt.axis('equal')

In [ ]:
treatment_lasers = imageio.imread(treatment_lasers_filepath)

In [ ]:
horizontal_line = (
    (treatment_lasers[:,:,1] == 255) &
    (treatment_lasers[:,:,0] == 0) & 
    (treatment_lasers[:,:,2] == 0)
)
plt.imshow(horizontal_line)

In [ ]:
vertical_line = (
    (treatment_lasers[:,:,2] == 255) &
    (treatment_lasers[:,:,1] == 0) & 
    (treatment_lasers[:,:,0] == 0)
)
plt.imshow(vertical_line)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(treatment_lasers[:,:,2])
plt.colorbar()
plt.axis('equal')

In [ ]:
# plt.pcolormesh?

In [ ]:
axes = create_axes(treatment_lasers)
y, x = axes[0][-1::-1], axes[1]

plt.figure(figsize=(10,10))
plt.pcolormesh(x, y, treatment_lasers[:,:,0])
plt.axis('equal')

In [ ]:
xx, yy = np.meshgrid(x, y)

x_hrz, y_hrz = xx[horizontal_line], yy[horizontal_line]
x_vert, y_vert = xx[vertical_line], yy[vertical_line]

hrz = linregress(x_hrz, y_hrz)
vert = linregress(x_vert, y_vert)

hrz_angle = np.arctan(hrz.slope) * 180 / np.pi
vert_angle = np.arctan(vert.slope) * 180 / np.pi

In [ ]:
hrz_angle

In [ ]:
vert_angle

In [ ]:
hrz_angle_from_vert = np.arctan(-1/vert.slope) * 180 / np.pi

average_hrz_angle = (hrz_angle_from_vert + hrz_angle) / 2
average_hrz_angle

In [ ]:
iso_x = -(vert.intercept - hrz.intercept) / (vert.slope - hrz.slope)
iso_y = iso_x * vert.slope + vert.intercept

In [ ]:
new_horz_m = np.tan(np.pi * average_hrz_angle/180)
new_vert_m = -1 / new_horz_m

new_horz_c = iso_y - new_horz_m * iso_x
new_vert_c = iso_y - new_vert_m * iso_x

In [ ]:
max_y = np.max(yy)
min_y = np.min(yy)

vert_pair_x = [
    (min_y - new_vert_c)/new_vert_m, 
    (max_y - new_vert_c)/new_vert_m
]
vert_pair_y = [min_y, max_y]

max_x = np.max(xx)
min_x = np.min(xx)

hrz_pair_x = [min_x, max_x]
hrz_pair_y = [
    min_x * new_horz_m + new_horz_c,
    max_x * new_horz_m + new_horz_c,
]


In [ ]:
plt.figure(figsize=(13,13))
plt.contourf(xx, yy, treatment_dose, 20)
plt.colorbar()

plt.plot(vert_pair_x, vert_pair_y)
plt.plot(hrz_pair_x, hrz_pair_y)
plt.plot(iso_x, iso_y, 'o', markeredgecolor='k', markersize=20)

plt.axis('equal')

In [ ]:
p1 = np.array([vert_pair_x[1], vert_pair_y[1]])
p2 = np.array([vert_pair_x[0], vert_pair_y[0]])
p3 = np.vstack([np.ravel(xx), np.ravel(yy)]).T

vert_d = np.cross(p2 - p1, p3 - p1) / np.linalg.norm(p2 - p1)
x_rotated = np.reshape(vert_d, np.shape(xx))

x_rotated

In [ ]:
p1 = np.array([hrz_pair_x[0], hrz_pair_y[0]])
p2 = np.array([hrz_pair_x[1], hrz_pair_y[1]])
p3 = np.vstack([np.ravel(xx), np.ravel(yy)]).T

horz_d = np.cross(p2 - p1, p3 - p1) / np.linalg.norm(p2 - p1)
y_rotated = np.reshape(horz_d, np.shape(yy))

y_rotated

In [ ]:
num_contours = 20

diff = np.ceil(np.nanmax(treatment_dose) / ((num_contours - 1) * 10)) * 10
levels = np.arange(0, diff * num_contours, diff).astype(int)
levels

In [ ]:
plt.figure(figsize=(13,13))
plt.contourf(x_rotated, y_rotated, treatment_dose, levels=levels)
plt.colorbar()

cs = plt.contour(x_rotated, y_rotated, filtered_dose, levels=levels, colors='k', alpha=1)
plt.clabel(cs, inline=1)


plt.plot([0, 0], [-90, 50])
plt.plot([-60, 60], [0, 0])
plt.plot(0, 0, 'o', markeredgecolor='k', markersize=20)

plt.axis('equal')

In [ ]:
x, y, z = xyz_axes_from_dataset(dicom_dose_dataset)  # pylint: disable=invalid-name
dose = dose_from_dataset(dicom_dose_dataset, reshape=False)

interpolation = RegularGridInterpolator((z, y, x), dose)

interpolated = interpolation(
    np.vstack([np.ravel(np.zeros_like(y_rotated)), -np.ravel(y_rotated + 20), np.ravel(x_rotated)]).T)

interpolated = np.reshape(interpolated, np.shape(y_rotated)) * 100

In [ ]:
plt.figure(figsize=(13,13))
plt.contourf(x_rotated, y_rotated, interpolated, levels=levels)
plt.colorbar()

plt.contour(x_rotated, y_rotated, interpolated, levels=levels, colors='k', alpha=0.4)


plt.plot([0, 0], [-90, 50])
plt.plot([-60, 60], [0, 0])
plt.plot(0, 0, 'o', markeredgecolor='k', markersize=20)

plt.axis('equal')

In [ ]:
diff = filtered_dose - interpolated

trim_margin = 100
trim_ref = (slice(trim_margin,-trim_margin), slice(trim_margin,-trim_margin))

max_diff = np.nanmax(np.abs(diff[trim_ref]))

plt.figure(figsize=(13,13))
plt.contourf(x_rotated[trim_ref], y_rotated[trim_ref], diff[trim_ref], 40, cmap='seismic', vmin=-max_diff, vmax=max_diff)
plt.colorbar()

In [ ]:
diff = treatment_dose - interpolated
diff[diff<-max_diff] = -max_diff
diff[diff>max_diff] = max_diff

trim_margin = 100
trim_ref = (slice(trim_margin,-trim_margin), slice(trim_margin,-trim_margin))

plt.figure(figsize=(13,13))
plt.contourf(x_rotated[trim_ref], y_rotated[trim_ref], diff[trim_ref], 40, cmap='seismic', vmin=-max_diff, vmax=max_diff)
plt.colorbar(label='Dose Difference [measured - monaco] (cGy)')

plt.xlabel('Patient Right/Left (mm)')
plt.ylabel('Patient Post/Ant (mm)')

In [ ]:
point_x = 0
point_y = -0.5
distance = 0.5

x_near = np.abs(x_rotated - point_x) < distance
y_near = np.abs(y_rotated - point_y) < distance

near_ref = x_near & y_near

num_points = np.sum(near_ref)
np.mean(filtered_dose[near_ref])